In [2]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
from datatools.maneger import DataManager
# from datatools.analyzer import clean_text
from error_tools import *

In [3]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [4]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                sys_utt.append( clean_text( ut.utt+"。" ))
                # sys_utt.append(ut.utt)
            else:   
                sys_utt.append(ut.utt)
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [13]:
n=3
modelM = DataManager("../models/utterance/")
model_name = "KLM_phrase_nucc_wiki_n={0}_noun1.pickle".format(n)
lm = modelM.load_data(model_name)

success load : ../models/utterance/KLM_phrase_nucc_wiki_n=3_noun1.pickle


In [16]:
mecab_analyzer = Analyzer(tokenizer_)
# 一文ずつしか送られて来ない読み
def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    if isinstance(sentences, str):
        # df = mecab_analyzer.analyze_with_dataframe(sentences)
        # words = []
        # for txt, pos in zip(df.surface, df.part_of_speech):
        #     if "名詞" in pos:
        #         words.append(pos)
        #     else:
        #         words.append(txt)
        # return words
        sentences = [sentences]

    for sen in sentences:
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        for txt, pos in zip(df.surface, df.part_of_speech):
            if "名詞" in pos:
                words.append(pos)
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen

def sentence2pos_mecab(sentences):
    normalize_sen = []
    if isinstance(sentences, str):
        sentences = [sentences]
    for sen in sentences:
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        for txt, pos in zip(df.surface, df.part_of_speech):
            words.append(pos)
        normalize_sen.append(words)
    return normalize_sen

In [ ]:

mecab_analyzer = Analyzer(tokenizer_)

def last_pos(pos_split):
    last = len(pos_split) - 1 
    for i, _ in enumerate(pos_split):
        if pos_split[last-i] != "*":
            return "["+pos_split[last-i]+"]"
def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    for sen in tqdm(sentences):
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        for txt, pos in zip(df.surface, df.part_of_speech):
            pos_split = pos.split("-")
            if pos_split[1] == "固有名詞" or pos_split[1] == "一般":
                # words.append(pos)
                words.append(last_pos(pos_split))
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen

In [51]:
import math
import pprint


def sentence2score(sentence, lm, N):
    # filled = fill_SYMBOL( sentence2normalize_independent(sentence) )
    filled = fill_SYMBOL_ONE( sentence2normalize_noun_mecab(sentence) )
    # filled = fill_SYMBOL( sentence2morpheme(sentence) )
    filled_pos = fill_SYMBOL_ONE( sentence2pos_mecab(sentence) )
    # print(filled)
    # print(filled_pos)
    ngram_text = []
    ngram_pos = []
    for L, P in zip(filled,filled_pos):
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
            ngram_pos.append(P[i:i+N])
    # pprint.pprint(ngram_text)
    all_score = 0
    function_score = 0
    # デフォルトで1
    function_num = 1

    under = 1 / (1000*len( lm.vocab.counts ) )

    for ngram, pgram in zip(ngram_text, ngram_pos):
        context = (ngram[:-1])
        context_pos = pgram[:-1]
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + under
        log_score = math.log2(score)
        # print("context : {0}|{1} ->".format(context, ngram[-1:]), log_score)

        if "助動詞" in context_pos[1] or "助詞" in context_pos[1] or "助動詞" in context_pos[0] or "助詞" in context_pos[0]:
            # print("\tcontext : {0}| ->".format(context), log_score)
            function_score += log_score
            function_num += 1

        all_score += log_score
    # print(all_score/len(ngram_text))
    return all_score/len(ngram_text)
    # return function_score/function_num

In [24]:
sentence = "最近とても暑いですから。"
# sentence = "魚はおいしいんですか？？海は素晴らしいですね。"
sentence = "魚はおいんですか？？海は素晴らしいですね。"
sentence2score(sentence, lm, N=n)

-6.536978645638182

In [52]:
y_scores = []
for utt in tqdm(sys_utt):
    y_scores.append(sentence2score(utt, lm, N=n))

100%|██████████| 1386/1386 [00:03<00:00, 412.15it/s]


In [36]:
def f1_score_plain(test, pred):
    return f1_score(y_true=test, y_pred=pred)

def recall_score_plain(test, pred):
    return recall_score(y_true=test, y_pred=pred)

In [53]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
import numpy as np

base = -5
epoch = 1000
# y_pred = np.zeros(len(y)) + 1
y_pred = np.zeros(len(y))
max_f1 = 0

print(model_name)
for e in tqdm(range(epoch)):
    y_pred = np.zeros(len(y))

    for i, score_ in enumerate(y_scores):
        # border 未満をエラーでとする
        border = (base - 0.1*e)
        if score_ < border :
            y_pred[i] = 1
    f1 = f1_score_plain(y, y_pred)
    if f1 > max_f1:
        max_f1 = f1
        print("f1 score:{0}, border:{1}".format(max_f1, border))

  6%|▌         | 57/1000 [00:00<00:01, 566.25it/s]

KLM_phrase_nucc_wiki_n=3_noun1.pickle
f1 score:0.013961605584642232, border:-5.0
f1 score:0.014625228519195614, border:-5.1
f1 score:0.015355086372360846, border:-5.2
f1 score:0.016129032258064516, border:-5.3
f1 score:0.01663201663201663, border:-5.4
f1 score:0.017204301075268814, border:-5.5
f1 score:0.01809954751131222, border:-5.6
f1 score:0.018518518518518517, border:-7.1
f1 score:0.01904761904761905, border:-7.2
f1 score:0.02040816326530612, border:-7.300000000000001
f1 score:0.0213903743315508, border:-7.4
f1 score:0.022598870056497175, border:-7.5
f1 score:0.024096385542168676, border:-7.6
f1 score:0.02547770700636943, border:-7.7
f1 score:0.0273972602739726, border:-7.800000000000001
f1 score:0.028368794326241138, border:-7.9
f1 score:0.03053435114503817, border:-8.0
f1 score:0.031496062992125984, border:-8.1
f1 score:0.034482758620689655, border:-8.2
f1 score:0.03636363636363636, border:-8.3


100%|██████████| 1000/1000 [00:01<00:00, 701.60it/s]


In [54]:
y_pred2 = np.zeros(len(y))

for i, score_ in enumerate(y_scores):
    if score_ < -8 :
        y_pred2[i] = 1

In [55]:
score(y, y_pred2)

confusion matrix = 
 [[1257  122]
 [   5    2]]
accuracy =  0.9083694083694084
precision =  0.016129032258064516
recall =  0.2857142857142857
f1 score =  0.03053435114503817
